# Fine-tuning Test

In [14]:
from datasets import *
import datasets

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, get_peft_model, LoraConfig
import evaluate


# dataset = load_dataset('lmsys/toxic-chat', 'toxicchat0124').shuffle()
# dataset = dataset.rename_column("user_input", "text")
# dataset = dataset.rename_column("jailbreaking", "label")
# dataset = dataset.remove_columns("conv_id")
# dataset = dataset.remove_columns("model_output")
# dataset = dataset.remove_columns("human_annotation")
# dataset = dataset.remove_columns("toxicity")
# dataset = dataset.remove_columns("openai_moderation")

# dataset = load_dataset('jackhhao/jailbreak-classification').shuffle()
# dataset = dataset.rename_column("prompt", "text")
# dataset = dataset.rename_column("type", "label")
# dataset = dataset.cast_column("label", datasets.ClassLabel(names=["benign", "jailbreak"]))

# dataset = load_dataset('deepset/prompt-injections').shuffle()

# dataset = load_dataset('xTRam1/safe-guard-prompt-injection').shuffle()

# dataset["test"][0]


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/shieldgemma-2b")
model = AutoModelForCausalLM.from_pretrained(
    "google/shieldgemma-2b",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

chat = [{"role": "user", "content": "Create 20 paraphrases of I hate you"}]

guideline = "\"No Harassment\": The prompt shall not contain or seek generation of content that is malicious, intimidating, bullying, or abusive content targeting another individual (e.g., physical threats, denial of tragic events, disparaging victims of violence)."
inputs = tokenizer.apply_chat_template(chat, guideline=guideline, return_tensors="pt", return_dict=True).to(model.device)

with torch.no_grad():
  logits = model(**inputs).logits

# Extract the logits for the Yes and No tokens
vocab = tokenizer.get_vocab()
selected_logits = logits[0, -1, [vocab['Yes'], vocab['No']]]

# Convert these logits to a probability with softmax
probabilities = torch.softmax(selected_logits, dim=0)

# Return probability of 'Yes'
score = probabilities[0].item()
print(score)  # 0.7310585379600525
